## 练习对 Spark RDD 的操作

大家联系的时候也可以用 pyspark shell

### 初始化 pyspark 环境

In [2]:
#导包
from pyspark import SparkConf, SparkContext  

In [3]:
# 创建 spark 上下文（context）， 并配置
conf = SparkConf().setMaster("local").setAppName("practice_RDD")  # 当前阶段 setMaster 就填 local 本地模式， appName 随便写
sc = SparkContext(conf = conf)  # sc 这个变量是使用 Spark 的入口， 如果使用 shell， shell 会自动帮你把这两部做了

### 常用的创建 RDD 的几种方式
1. 读文件
2. 从 python 的 list 转化
3. 直接创建

In [6]:
# 读文件，按行读取
# readFile = sc.textFile('')

In [7]:
# 从 python 的 list 转化
l = [1, 3, 4, 6, "asd"]
fromPythonList = sc.parallelize(l)
fromPythonList.collect()

[1, 3, 4, 6, 'asd']

In [8]:
# 直接创建
sc.range(0,10).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### 注意 collect 前后的类型
`没 collect 的时候是 RDD， collect 之后就是 python 的 list 了`

In [12]:
type(sc.range(0,10))

pyspark.rdd.PipelinedRDD

In [13]:
type(sc.range(0,10).collect())

list

### 几个常用的操作
* map
* flatmap
* filter
* reduce
* reduceByKey
* collect

#### Map

In [ ]:
#初始的 RDD
rdd = sc.range(0,10)
rdd.collect() 

In [17]:
# map
rddAfterMap = rdd.map(lambda x : x + 99)
rddAfterMap.collect()

[99, 100, 101, 102, 103, 104, 105, 106, 107, 108]

In [18]:
 # 初始的 RDD
rdd2 = sc.parallelize([(1, 5),(3, 6),(4, 99),(7, 101)])
rdd2.collect

[(1, 5), (3, 6), (4, 99), (7, 101)]

In [21]:
rdd2.map(lambda x : x[0] + x[1]).collect()

[6, 9, 103, 108]

#### FlatMap (体会 map flap 的区别)

In [33]:
rdd2.map(lambda x : x).collect() # map不变

[(1, 5), (3, 6), (4, 99), (7, 101)]

In [32]:
rdd2.flatMap(lambda x : x).collect()  # flatten

[1, 5, 3, 6, 4, 99, 7, 101]

In [36]:
rdd2.map(lambda x : [x[0] + 1, x[1] + 1]).collect() #map

[[2, 6], [4, 7], [5, 100], [8, 102]]

In [35]:
rdd2.flatMap(lambda x : [x[0] + 1, x[1] + 1]).collect()  # flatten

[2, 6, 4, 7, 5, 100, 8, 102]

#### Filter (返回一个 boolean )

In [44]:
# rdd2  [(1, 5),(3, 6),(4, 99),(7, 101)]
rdd2.filter(lambda x : x[0] & 1 ).collect()  # x[0] 是奇数？

[(1, 5), (3, 6), (7, 101)]

#### Reduce (输入的参数一定是两个)

In [47]:
rdd3 = sc.range(1, 10)
rdd3.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [49]:
rdd3.reduce(lambda x,y : x + y)  # 求和，返回值是 int

45

#### ReduceByKey

In [54]:
 rdd4 = sc.parallelize([(1, 5),(1, 55), (3, 6), (3, 66), (4, 9), (4, 99), (7, 101)])
# rdd4 = sc.parallelize([[1, 5],[1, 55], [3, 6], [3, 66], [4, 9], [4, 99], [7, 101]]) 也可以

In [55]:
rdd4.reduceByKey(lambda x, y : x + y).collect()

[(1, 60), (3, 72), (4, 108), (7, 101)]

#### Collect 和 Lazy Evaluation (需要去 IDE 演示)

In [58]:
# rdd3 [1, 2, 3, 4, 5, 6, 7, 8, 9]
def f(x):
    print("Mapping Once")
    return x + 1 
rdd3AfterMap = rdd3.map(f) #此时 RDD 并没有真正被操作

In [63]:
rdd3AfterMap.collect() # 此处应该还会输出很多"Mapping Once"， 但是 jupyter 的问题没输出

[2, 3, 4, 5, 6, 7, 8, 9, 10]

#### 还有很多 API 例如 Fold.. GroupByKey ...SortByKey

### DAG

In [65]:
rdd3AfterMap.toDebugString()

b'(1) PythonRDD[48] at collect at <ipython-input-59-e6bd2b5a4286>:1 []\n |  ParallelCollectionRDD[30] at parallelize at PythonRDD.scala:195 []'